# Getting Google Trends Data Using Pytrends

Pytrends is a Python library that allows you to easily access Google Trends data. Google Trends provides insights into what people are searching for on Google. Pytrends simplifies the process of fetching and analyzing this data programmatically. In this tutorial, we will walk through the installation process and demonstrate how to use Pytrends to retrieve and visualize Google Trends data.

## 1. Installation

First, make sure you have Python installed on your system. You can download and install Python from [python.org](https://www.python.org/downloads/).

You can install Pytrends using pip, the Python package manager:

In [1]:
# !pip install pytrends
# ! pip install pytrends --upgrade
# ! pip install git+https://github.com/GeneralMills/pytrends

Once Pytrends is installed, you can import it into your Python script or interactive session:

In [8]:
from pytrends.request import TrendReq
import requests

In [9]:
session = requests.Session()
session.get('https://trends.google.com')
cookies_map = session.cookies.get_dict()
nid_cookie = cookies_map['NID']

In [10]:
# version(pytrends)

## 2. Retrieving Google Trends Data

### Initializing Pytrends

To start retrieving Google Trends data, create a `TrendReq` object:

In [11]:
# pytrends = TrendReq(retries=5, backoff_factor=0.1)
# pytrends = TrendReq(requests_args={'headers': {'Cookie': f'NID={nid_cookie}'}})
pytrends = TrendReq()

### Searching for Keywords

You can search for the popularity of specific keywords over time:

In [12]:
keywords = ["Python", "Data Science", "Machine Learning"]
pytrends.build_payload(keywords, timeframe='today 5-y')

### Retrieving Interest Over Time

You can retrieve the interest over time for the specified keywords:

In [13]:
interest_over_time_df = pytrends.interest_over_time()
print(interest_over_time_df.head())

TooManyRequestsError: The request failed: Google returned a response with code 429

In [ ]:
interest_over_time_df

### Retrieving Related Queries

You can also retrieve related queries for a specific keyword:

In [ ]:
related_queries = pytrends.related_queries()
print(related_queries)

## 3. Visualizing Trends Data

### Using Matplotlib for Visualization

You can visualize Google Trends data using popular plotting libraries like Matplotlib:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
interest_over_time_df.plot()
plt.title('Interest Over Time')
plt.xlabel('Date')
plt.ylabel('Interest')
plt.grid(True)
plt.show()

## 4. Advanced Usage

### Geographic Trends

You can retrieve geographic trends for specific keywords:

In [ ]:
pytrends.build_payload(keywords, geo='US')
geo_interest = pytrends.interest_by_region()
print(geo_interest)

In [ ]:
# let's try for Germany
pytrends.build_payload(keywords, geo='DE')
geo_interest = pytrends.interest_by_region()
print(geo_interest)

### Category Trends

You can retrieve trends for specific categories as well. Category refer to broad topical classifications like 'Music & Audio'. Here are all the categories: https://github.com/pat310/google-trends-api/wiki/Google-Trends-Categories

In [ ]:
pytrends.build_payload(keywords, cat=0) # 0 is for all
category_interest = pytrends.interest_by_region()
print(category_interest)

In [ ]:
pytrends.build_payload(keywords, cat=231) # 231 is for 'Engineering & Technology'
category_interest = pytrends.interest_by_region()
print(category_interest)

### Real-time Trends

You can retrieve real-time trends:

In [ ]:
pytrends.build_payload(keywords, timeframe='now 1-H')
realtime_interest = pytrends.interest_over_time()
print(realtime_interest)

### Spatial Plot of the Trends

In [ ]:
import plotly.express as px

In [ ]:
keywords = ['barbie', 'oppenheimer']
pytrends.build_payload(keywords, geo='US',
                      timeframe='2023-07-01 2023-08-01',
                      cat = 3 # 'Arts & Entertainment'
                      ) 
geo_interest = pytrends.interest_by_region()
print(geo_interest)

In [ ]:
geo_interest['diff'] = geo_interest['barbie'] - geo_interest['oppenheimer']
geo_interest = geo_interest.reset_index()
geo_interest

In [ ]:
# let's get the state abbreviations

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
geo_interest['state_code'] = [us_state_to_abbrev[i] for i in geo_interest['geoName']]

In [ ]:
geo_interest

In [ ]:
fig = px.choropleth(geo_interest,
                    locations='state_code',
                    color='diff',
                    color_continuous_scale='spectral_r',
                    hover_name='state_code',
                    locationmode='USA-states',
                    scope='usa')
fig.add_scattergeo(
    locations=geo_interest['state_code'],
    locationmode="USA-states", 
    text=geo_interest['state_code'],
    mode='text',
)
fig.update_layout(
    title_x = 0.5,
    title_y = 0.95,
    title_text='Preference for Barbie over Oppenheimer by US States',
    margin=dict(t=10, b=0, r=0, l=0),  # Set b (bottom) margin to 0
    autosize=True,
    # width=1300,
    # height=700,
)

fig.show()
fig.write_image('barbenheimer_interest.pdf')